# Input Feeding 기법

In [1]:
import torch
import torch.nn as nn

In [2]:
class InputFeedingDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):   # output_size = vocab_size
        super(InputFeedingDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, input_size)
        self.lstm = nn.LSTM(input_size + hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden, context):
        embedded = self.embedding(input).unsqueeze(0)  # (1, batch_size, input_size)
        lstm_input = torch.cat((embedded, context.unsqueeze(0)), dim=2)  # (1, batch_size, input_size + hidden_size)
        output, hidden = self.lstm(lstm_input, hidden)  # output: (1, batch_size, hidden_size), hidden: (1, batch_size, hidden_size)
        output = self.fc(output.squeeze(0))  # (batch_size, output_size)
        return output, hidden

In [3]:
decoder = InputFeedingDecoder(input_size=10, hidden_size=20, output_size=30)
hidden = (torch.zeros(1, 1, 20), torch.zeros(1, 1, 20))  # (num_layers, batch_size, hidden_size)
context = torch.zeros(1, 20)    
input_token = torch.tensor([5]) # Example input token (index in the vocabulary) / 입력값

output, hidden = decoder(input_token, hidden, context)
output.shape, hidden[0].shape


(torch.Size([1, 30]), torch.Size([1, 1, 20]))